In [44]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import \
     (Stepwise,
      sklearn_selected,
      sklearn_selection_path)
from l0bnb import fit_path

In [45]:
data = load_data("College")
data.head(5)
np.isnan(data['Grad.Rate']).sum()

0

In [46]:
design = MS(['Accept', 'Expend']).fit(data)
Y = np.array(data['Grad.Rate'])
D = design.fit_transform(data)
D = D.drop('intercept', axis=1)
X = np.asarray(D)

X

array([[ 1232,  7041],
       [ 1924, 10527],
       [ 1097,  8735],
       ...,
       [ 1915,  8323],
       [ 2453, 40386],
       [ 1855,  4509]], dtype=int64)

In [47]:
Xs = X - X.mean(0)[None,:]
X_scale = X.std(0)
Xs = Xs / X_scale[None,:]
lambdas = 10**np.linspace(8, -2, 100) / Y.std()
soln_array = skl.ElasticNet.path(Xs,
                                 Y,
                                 l1_ratio=0.,
                                 alphas=lambdas)[1]
soln_array.shape

C:\Users\bruce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:683: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\bruce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1779384.4969163411, tolerance: 355.87690000000003
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\bruce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:683: UserWarning: Coordinate 

(2, 100)

In [48]:
soln_path = pd.DataFrame(soln_array.T,
                         columns=D.columns,
                         index=-np.log(lambdas))
soln_path.index.name = 'negative log(lambda)'
soln_path

,Accept,Expend
negative log(lambda),,
-15.577712,1.983660e-07,0.000001
-15.345128,2.503094e-07,0.000001
-15.112543,3.158546e-07,0.000002
-14.879959,3.985633e-07,0.000002
-14.647375,5.029298e-07,0.000003
...,...,...
6.517801,3.256267e-01,6.650444
6.750386,3.254704e-01,6.652499
6.982970,3.253463e-01,6.654129


In [49]:
ridge = skl.ElasticNet(alpha=lambdas[59], l1_ratio=0)
scaler = StandardScaler(with_mean=True,  with_std=True)
pipe = Pipeline(steps=[('scaler', scaler), ('ridge', ridge)])
pipe.fit(X, Y)

C:\Users\bruce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.121e+05, tolerance: 2.290e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('scaler', StandardScaler()),
                ('ridge', ElasticNet(alpha=6.393201978970811, l1_ratio=0))])

In [50]:
validation = skm.ShuffleSplit(n_splits=1,
                              test_size=0.5,
                              random_state=0)
ridge.alpha = 0.01
results = skm.cross_validate(ridge,
                             X,
                             Y,
                             scoring='neg_mean_squared_error',
                             cv=validation)
-results['test_score']

C:\Users\bruce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.852e+04, tolerance: 1.157e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


array([251.0608875])

In [53]:
K = 5
kfold = skm.KFold(K,
                  random_state=0,
                  shuffle=True)
param_grid = {'ridge__alpha': lambdas}
grid = skm.GridSearchCV(pipe, 
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
grid.fit(X, Y)
grid.best_params_['ridge__alpha']
print("optimal lambda:",grid.best_params_['ridge__alpha'])
print("MSE: ",-grid.best_score_)

C:\Users\bruce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.054e+04, tolerance: 1.811e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\bruce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality

optimal lambda: 0.024069995050683645
MSE:  251.37569932071014


In [52]:
lassoCV = skl.ElasticNetCV(n_alphas=100, 
                           l1_ratio=1,
                           cv=kfold)
pipeCV = Pipeline(steps=[('scaler', scaler),
                         ('lasso', lassoCV)])
pipeCV.fit(X, Y)
tuned_lasso = pipeCV.named_steps['lasso']

print("optimal lambda: ",tuned_lasso.alpha_)
print("MSE: ",np.min(tuned_lasso.mse_path_.mean(1)))
print(tuned_lasso.coef_,"no efficients set to zero by the Lasso")

optimal lambda:  0.20463511375367993
MSE:  251.37567813541108
[0.14293029 6.47841645] no efficients set to zero by the Lasso


The optimal lambda for Ridge regression is much smaller than the one for Lasso regression. This suggests that Ridge regression requires less regularization to reach its best performance compared to Lasso.
A smaller lambda in Ridge regression means the penalty on the size of coefficients is relatively low, allowing the model to fit the data more flexibly.